# **모델 매개변수 최적화하기**

앞선 챕터들에서 모델과 데이터를 준비했습니다. 이제 데이터에 매개변수를 최적화하여 모델을 학습하고, 검증하고, 테스트할 차례입니다. 모델을 학습하는 과정으 반복적인 과정을 거칩니다. epoch이라고 흔히 말하는 각 반복 단계에서 모델은 출력을 추측하고, 추측과 정답 사이의 오류(loss)를 계산하고, 매개변수에 대한 오류의 도함수를 수집한 뒤, 경사하강법을 사용하여 이 파라미터들을 최적화(optimize)합니다.

## **기본(Pre-requisite) 코드**

이전 장의 코드를 가져오겠습니다.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork()

## **Hyperparameter**

하이퍼파라미터는 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수입니다. 서로 다른 하이퍼파라미터 값은 모델 학습과 수렴율(convergence rate)에 영향을 미칠 수 있습니다. ([하이퍼파라미터 튜닝 더 알아보기](https://tutorials.pytorch.kr/beginner/hyperparameter_tuning_tutorial.html))

학습 시에는 다음과 같은 하이퍼파라미터를 정의합니다.
- **epoch** - 데이터셋을 반복하는 횟수
- **batch size** - 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
- **learning rate** - 각 batch 혹은 epoch에서 모델의 매개변수를 조절하는 비율. 값이 작을수록 학습속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있습니다. 

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## **Optimization Loop**

하이퍼파라미터를 설정한 뒤에는 최적화 단계를 통해 모델을 학습하고 최적화할 수 있습니다. 최적화 단계의 각 반복(iteration)을 epoch라고 합니다.

하나의 epoch는 다음 두 부부으로 구성됩니다.
- **학습 단계(train loop)** - 학습용 데이터셋을 반복하고 최적의 매개변수로 수렴합니다.
- **검증/테스트 단계(validation/test loop)** - 모델 성능이 개선되고 있는지 확인하기 위해 테스트 데이터셋을 반복합니다.

학습 단계에서 일어나는 몇 가지 개념들을 간략히 살펴보겠습니다. 최적화 단계는 후에 나올 예정입니다.

## **Loss function**

학습용 데이터를 제공하면, 학습되지 않은 신경망은 정답을 제공하지 않을 확률이 높습니다. 손실 함수느 획득한 결과와 실제 값 사이의 틀린 정도(degree of dissimilarity)를 측정하며, 학습 중에 이 값을 최소화하려고 합니다. 주어진 데이터 샘플을 입력으로 계산한 예측과 정답(label)을 비교하여 손실을 계산합니다. 

일반적인 손실함수에는 회귀 문제(regression task)에 사용하는 `nn.MSELoss`(평균 제곱 오차)나 분류에 사용하는 `nn.NLLLoss`(Negative Log Likelihood), 그리고 `nn.LogSoftmax`와 `nn.NLLLoss`를 합친 `nn.CrossEntropyLoss` 등이 있습니다. 

모델의 출력 logit을 `nn.CrossEntropyLoss`에 전달하여 logit을 정규화하고 예측 오류를 계산합니다.

In [3]:
# 손실 함수를 초기화합니다.
loss_fn = nn.CrossEntropyLoss()

## **Optimizer**

최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정입니다. 최적화 알고리즘은 이 과정이 수행되는 방식(여기에서는 확률적 경상하강법(SGD; Stochastic Gradient Descent))을 정의합니다. 모든 최적화 절차(logit)는 `optimizer` 객체에 캡슐화(encapsulate)됩니다. 여기서는 SGD 옵티마이저를 사용하고 있으며, PyTorch에는 ADAM이나 RMSProp와 같은 다른 종류의 모델과 데이터에서 더 잘 작동하는 다양한 옵티마이저가 있습니다. 

학습하려는 모델의 매개변수와 학습률 하이퍼파라미터를 등록하여 옵티마이저를 초기화합니다.

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

학습 단계에서 최적화는 세 단계로 이루어집니다.
- `optimizer.zero_grad()`를 호출하여 모델 매개변수의 변화도를 재설정합니다. 기본적으로 변화도는 더해지기(add up) 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정합니다.
- `loss.backward()`를 호출하여 예측 손실(prediction loss)을 역전파합니다. PyTorch는 각 매개벼수에 대한 손실의 변화도를 저장합니다.
- 변화도를 계산한 뒤에는 `optimizer.step()`을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정합니다.

## **전체 구현**

최적화 코드를 반복하여 수행하는 `train_loop`와 테스트 데이터로 모델의 성능을 측정하는 `test_loop`를 정의했습니다.

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # prediction과 loss 계산
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # backprapagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

손실 함수와 옵티마이저를 초기화하고 `train_loop`와 `test_loop`에 전달합니다. 모델의 성능 향상을 알아보기 위해 자유롭게 epoch 수를 증가시켜 볼 수 있습니다.

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
--------------------------------------
loss: 2.172620 [    0/60000]
loss: 2.177859 [ 6400/60000]
loss: 2.117976 [12800/60000]
loss: 2.148351 [19200/60000]
loss: 2.096275 [25600/60000]
loss: 2.030796 [32000/60000]
loss: 2.085051 [38400/60000]
loss: 2.001935 [44800/60000]
loss: 2.004154 [51200/60000]
loss: 1.951130 [57600/60000]
Test Error: 
 Accuracy: 53.7%, Avg loss: 1.937160 

Epoch 2
--------------------------------------
loss: 1.956720 [    0/60000]
loss: 1.947445 [ 6400/60000]
loss: 1.827912 [12800/60000]
loss: 1.885595 [19200/60000]
loss: 1.765440 [25600/60000]
loss: 1.706828 [32000/60000]
loss: 1.758420 [38400/60000]
loss: 1.640265 [44800/60000]
loss: 1.655632 [51200/60000]
loss: 1.571076 [57600/60000]
Test Error: 
 Accuracy: 61.6%, Avg loss: 1.567236 

Epoch 3
--------------------------------------
loss: 1.618651 [    0/60000]
loss: 1.597743 [ 6400/60000]
loss: 1.435341 [12800/60000]
loss: 1.522757 [19200/60000]
loss: 1.384724 [25600/60000]
loss: 1.377697 [32000/60000]
l

In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork()

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
--------------------------------------
loss: 2.306897 [    0/60000]
loss: 0.553359 [ 6400/60000]
loss: 0.394523 [12800/60000]
loss: 0.502762 [19200/60000]
loss: 0.426639 [25600/60000]
loss: 0.432372 [32000/60000]
loss: 0.378134 [38400/60000]
loss: 0.520631 [44800/60000]
loss: 0.478595 [51200/60000]
loss: 0.526594 [57600/60000]
Test Error: 
 Accuracy: 83.7%, Avg loss: 0.444078 

Epoch 2
--------------------------------------
loss: 0.296570 [    0/60000]
loss: 0.351552 [ 6400/60000]
loss: 0.303195 [12800/60000]
loss: 0.389461 [19200/60000]
loss: 0.381743 [25600/60000]
loss: 0.365842 [32000/60000]
loss: 0.320844 [38400/60000]
loss: 0.507298 [44800/60000]
loss: 0.392264 [51200/60000]
loss: 0.487467 [57600/60000]
Test Error: 
 Accuracy: 85.5%, Avg loss: 0.393266 

Epoch 3
--------------------------------------
loss: 0.218889 [    0/60000]
loss: 0.329521 [ 6400/60000]
loss: 0.235537 [12800/60000]
loss: 0.324807 [19200/60000]
loss: 0.425427 [25600/60000]
loss: 0.343203 [32000/60000]
l